In [1]:
#Imports
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score,confusion_matrix
%matplotlib inline

#Leitura dos ficheiros
train = pd.read_csv('datasets/training_data.csv',encoding='cp1252')
test = pd.read_csv('datasets/test_data.csv',encoding='cp1252')

#Funções auxiliares
ordem = {'None':0, 'Low':1, 'Medium':2, 'High':3, 'Very_High':4}

def round_pred(pred,predictions):
    for n in pred:
        n = int(round(n))
        if n == 0: predictions.append('None')
        elif n == 1: predictions.append('Low')
        elif n == 2: predictions.append('Medium')
        elif n == 3: predictions.append('High')
        else: predictions.append('Very_High')

,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,AVERAGE_TEMPERATURE,AVERAGE_ATMOSP_PRESSURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,AVERAGE_PRECIPITATION
count,6812.000000,6812.000000,6812.000000,6812.000000,6812.000000,6812.000000,6812.000000,6812.0
mean,40.661010,25.637111,81.143952,16.193482,1017.388139,80.084190,3.058573,0.0
std,4.119023,33.510507,8.294401,5.163492,5.751061,18.238863,2.138421,0.0
min,30.500000,0.000000,46.400000,0.000000,985.000000,14.000000,0.000000,0.0
25%,37.600000,2.275000,75.400000,13.000000,1015.000000,69.750000,1.000000,0.0
50%,40.700000,12.200000,82.400000,16.000000,1017.000000,83.000000,3.000000,0.0
75%,43.500000,36.200000,87.400000,19.000000,1021.000000,93.000000,4.000000,0.0
max,55.900000,296.500000,112.000000,35.000000,1033.000000,100.000000,14.000000,0.0


In [2]:
#Tratamento de dados

#Feriados - Dias da semana - Estações
feriados = ['2018-01-01', '2018-02-13', '2018-03-30', '2018-04-01', '2018-04-25', '2018-05-01', '2018-05-31', 
            '2018-06-10', '2018-08-15', '2018-10-05', '2018-11-01', '2018-12-01', '2018-12-08', '2018-12-25',
            '2019-01-01', '2019-03-05', '2019-04-19', '2019-04-21', '2019-04-25', '2019-05-01', '2019-06-10', 
            '2019-06-20', '2019-08-15', '2019-10-05', '2019-11-01', '2019-12-01', '2019-12-08', '2019-12-25']

for index,row in train.iterrows():
    data = row[1].split(' ')[0]
    month = data.split('-')[1]
    if month in ['01','02','03']: train.at[index,'Estacao'] = 'Inverno'
    elif month in ['04','05','06']: train.at[index,'Estacao'] = 'Primavera'
    elif month in ['07','08','09']: train.at[index,'Estacao'] = 'Verao'
    else: train.at[index,'Estacao'] = 'Outono'
    dia = pd.Timestamp(data)
    train.at[index,'Dia'] = dia.day_name()
    if data in feriados: train.at[index,'Feriado'] = True
    else: train.at[index,'Feriado'] = False
        
for index,row in test.iterrows():
    data = row[1].split(' ')[0]
    month = data.split('-')[1]
    if month in ['01','02','03']: test.at[index,'Estacao'] = 'Inverno'
    elif month in ['04','05','06']: test.at[index,'Estacao'] = 'Primavera'
    elif month in ['07','08','09']: test.at[index,'Estacao'] = 'Verao'
    else: test.at[index,'Estacao'] = 'Outono'
    dia = pd.Timestamp(data)
    test.at[index,'Dia'] = dia.day_name()
    if data in feriados: test.at[index,'Feriado'] = True
    else: test.at[index,'Feriado'] = False
        
del train['AVERAGE_PRECIPITATION']
del test['AVERAGE_PRECIPITATION']
del train['city_name']
del test['city_name']
del train['record_date']
del test['record_date']
del train['AVERAGE_CLOUDINESS']
del test['AVERAGE_CLOUDINESS']
del train['AVERAGE_RAIN']
del test['AVERAGE_RAIN']
del train['AVERAGE_TEMPERATURE']
del test['AVERAGE_TEMPERATURE']

#train['AVERAGE_CLOUDINESS'].fillna(method ='pad',inplace=True)
#train['AVERAGE_CLOUDINESS'].fillna(method ='bfill',inplace=True)
#train['AVERAGE_RAIN'].fillna(method ='pad',inplace=True)
#train['AVERAGE_RAIN'].fillna(method ='bfill',inplace=True)

In [3]:
#Substituir outliers
q = train['AVERAGE_TIME_DIFF'].quantile(0.9)
train = train[train['AVERAGE_TIME_DIFF']<q]

In [3]:
#Preparação para colocar nos modelos
X = train.drop(['AVERAGE_SPEED_DIFF'],axis=1)
Y = train['AVERAGE_SPEED_DIFF'].to_frame()
Y_2 = train['AVERAGE_SPEED_DIFF'].map(ordem)
X_teste = test
#Y_test = test['AVERAGE_SPEED_DIFF'].to_frame()

luminosity = LabelEncoder()
#cloudiness = LabelEncoder()
#rain = LabelEncoder()
feriado = LabelEncoder()
day = LabelEncoder()
season = LabelEncoder()
luminosity_t = LabelEncoder()
#cloudiness_t = LabelEncoder()
#rain_t = LabelEncoder()
feriado_t = LabelEncoder()
day_t = LabelEncoder()
season_t = LabelEncoder()


X['LUMINOSITY_n'] = luminosity.fit_transform(X['LUMINOSITY'])
#X['AVERAGE_CLOUDINESS_n'] = cloudiness.fit_transform(X['AVERAGE_CLOUDINESS'])
#X['AVERAGE_RAIN_n'] = rain.fit_transform(X['AVERAGE_RAIN'])
X['Feriado_n'] = feriado.fit_transform(X['Feriado'])
X['Dia_n'] = day.fit_transform(X['Dia'])
X['Estacao_n'] = season.fit_transform(X['Estacao'])

X_teste['LUMINOSITY_n'] = luminosity_t.fit_transform(X_teste['LUMINOSITY'])
#X_test['AVERAGE_CLOUDINESS_n'] = cloudiness_t.fit_transform(X_test['AVERAGE_CLOUDINESS'])
#X_test['AVERAGE_RAIN_n'] = rain_t.fit_transform(X_test['AVERAGE_RAIN'])
X_teste['Feriado_n'] = feriado_t.fit_transform(X_teste['Feriado'])
X_teste['Dia_n'] = day_t.fit_transform(X_teste['Dia'])
X_teste['Estacao_n'] = season_t.fit_transform(X_teste['Estacao'])


X = X.drop(['LUMINOSITY','Dia','Feriado','Estacao'],axis=1)
X_teste = X_teste.drop(['LUMINOSITY', 'Dia','Feriado','Estacao'],axis=1)

In [6]:
#DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state= 2021)
dtc.fit(X,Y)
predictions = dtc.predict(X_test)
predictions

array(['None', 'Medium', 'None', ..., 'None', 'Very_High', 'Low'],
      dtype=object)

In [7]:
#Linear Regression
lr = LinearRegression()
lr.fit(X,Y_2)
pred = lr.predict(X_test)
predictions = []
round_pred(pred,predictions)
predictions

['None',
 'Low',
 'None',
 'High',
 'Low',
 'Medium',
 'Low',
 'Low',
 'Low',
 'Medium',
 'None',
 'Low',
 'Medium',
 'High',
 'High',
 'High',
 'None',
 'High',
 'Low',
 'None',
 'None',
 'Low',
 'Low',
 'None',
 'Medium',
 'Medium',
 'Medium',
 'None',
 'None',
 'Very_High',
 'Low',
 'Low',
 'Very_High',
 'Low',
 'Low',
 'Low',
 'Low',
 'High',
 'Medium',
 'None',
 'Low',
 'None',
 'High',
 'Very_High',
 'High',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Low',
 'Very_High',
 'Medium',
 'None',
 'Medium',
 'High',
 'Very_High',
 'Very_High',
 'None',
 'None',
 'Low',
 'Low',
 'Low',
 'Very_High',
 'None',
 'Very_High',
 'Low',
 'Medium',
 'Very_High',
 'None',
 'Low',
 'Low',
 'Low',
 'Very_High',
 'High',
 'Low',
 'None',
 'Medium',
 'Medium',
 'Low',
 'Low',
 'Very_High',
 'None',
 'None',
 'Low',
 'None',
 'Medium',
 'None',
 'High',
 'Medium',
 'None',
 'None',
 'High',
 'Medium',
 'None',
 'Very_High',
 'None',
 'None',
 'Very_High',
 'Medium',
 'Low',
 'None',
 'None',
 '

In [11]:
#Logistic Regression
logr = LogisticRegression(max_iter=10000)
X_train,X_test,y_train,y_test = train_test_split(X,Y_2, test_size=0.2, random_state=2021)
logr.fit(X_train,y_train)
pred = logr.predict(X_test)
predictions = []
#round_pred(pred,predictions)
accuracy_score(y_test, pred)


0.7681584739545121

In [16]:
#LinearSVC
model = LinearSVC(dual=False)
model.fit(X,Y)

predictions = model.predict(X_test)
predictions

C:\Users\tiago\anaconda3\envs\DAA\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['None', 'Low', 'None', ..., 'None', 'Very_High', 'Medium'],
      dtype=object)

In [4]:
#GridSearch
param_grid = {'C': [0.1,1,10,100,1000], 'gamma': [1,0.1,0.01,0.001,0.0001],'kernel': ['rbf']}
grid = GridSearchCV(SVC(random_state=2021),param_grid,refit=True,verbose=3)


#X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size=0.2, random_state=2021)
grid.fit(X,Y)

predictions = grid.predict(X_teste)
#accuracy_score(y_test, predictions)
#predictions

Fitting 5 folds for each of 25 candidates, totalling 125 fits


C:\Users\tiago\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.323 total time=   7.6s


C:\Users\tiago\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.323 total time=   5.2s


...

C:\Users\tiago\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 4/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.784 total time=   1.6s


C:\Users\tiago\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 5/5] END ..C=1000, gamma=0.0001, kernel=rbf;, score=0.789 total time=   1.5s


C:\Users\tiago\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [5]:
submission = open("submissions/submission18.csv", "w", newline='')
writer = csv.writer(submission)
writer.writerow(['RowId', 'Speed_Diff'])
rowId = list(range(0, 1500))
for i in range(1500):
    writer.writerow([rowId[i]+1, predictions[i]])

submission.close()

In [34]:
X

,AVERAGE_FREE_FLOW_SPEED,AVERAGE_TIME_DIFF,AVERAGE_FREE_FLOW_TIME,AVERAGE_TEMPERATURE,AVERAGE_HUMIDITY,AVERAGE_WIND_SPEED,Estação,LUMINOSITY_n,Feriado_n,Dia_n
0,41.5,11.5,71.4,15.0,100.0,3.0,Inverno,1,0,4
1,41.7,48.3,87.4,21.0,53.0,5.0,Outono,1,0,0
2,38.6,38.4,85.2,26.0,61.0,4.0,Verao,1,0,3
3,37.4,61.0,94.1,18.0,48.0,4.0,Verao,1,0,5
4,41.6,50.4,77.0,15.0,82.0,10.0,Outono,1,0,4
...,...,...,...,...,...,...,...,...,...,...
6807,41.2,3.7,86.9,9.0,71.0,4.0,Inverno,1,0,2
6808,38.9,51.9,76.9,17.0,63.0,6.0,Primavera,1,0,2
6809,39.6,0.0,89.1,15.0,39.0,4.0,Outono,0,0,5
6810,41.6,0.0,85.5,8.0,81.0,1.0,Inverno,0,0,6
